# <font class="markdown-google-sans" size=7>**HiFi GAN BWE 📈⚙️🎶**</font>

> ⚙️ <i>[...] unoffical implementation of the **HiFi-GAN+ model for audio bandwidth extension**, from the paper [“<font color=LightBlue>**B**</font>and<font color=LightBlue>**w**</font>idth <font color=LightBlue>**E**</font>xtension is All You Need”](https://doi.org/10.1109/ICASSP39728.2021.9413575) by Jiaqi Su, Yunyun Wang, Adam Finkelstein, and Zeyu Jin.</i>

> 📈 <i>The model **takes a band-limited audio signal** (usually `8`/`16`/`24`kHz) and **attempts to reconstruct the high frequency components needed to restore a full-band signal at** <u>48kHz</u><font color=yellow><sup>1</sup></font>.</i>

> 🎶 <i>This is useful for **upsampling low-rate outputs from upstream tasks like text-to-speech, voice conversion**, etc. or enhancing audio that was filtered to remove high frequency noise.</i>

<br><hr><br>

### Features:
- URL support 🔗
- **Multiple output target samplerates**<font color=yellow><sup>1</sup></font> 📈

<br><hr><br>

<a href="https://github.com/brentspell/hifi-gan-bwe"><img src="https://img.shields.io/badge/github-open-lightgray?logo=github&logoColor=white&style=for-the-badge"></a>

# <div class="markdown-google-sans"><font color=lime>**1.**</font> Re/install requirements 📥</div>

In [ ]:
#@markdown > ℹ️ Runtime does not need to be restarted.
import os
import time
import torch
import locale
import warnings
import numpy as np
from time import time
from pathlib import Path
from shutil import rmtree
from zipfile import ZipFile
from datetime import timedelta
from rich import print as rprint
from mimetypes import guess_type
from scipy.io.wavfile import write
from requests import Session, utils

try:
	from google.colab import files, runtime
	is_colab = True
except ImportError:
	is_colab = False

GPU = torch.cuda.device_count()
warnings.filterwarnings("ignore")
locale.getpreferredencoding = lambda: "UTF-8"

#-=-=-=-#
# Metadata

__title__  = "HiFi GAN BWE"
__author__ = "kubinka0505"
__date__   = "04th December 2023"
__title__  = __title__.replace(" ", "_")

#-=-=-=-#
# Modules & packages

requirements = {
	"modules":  ["mutagen", "hifi_gan_bwe"],
	"packages": ["sox"]
}

if GPU:
	GPU_Name = torch.cuda.get_device_name(torch.cuda.current_device())
	rprint("[b #00CC00]GPU Available![/] [#A0A0A0]({0})[/]".format(GPU_Name))
else:
	rprint("[#FFCC00][b]GPU Unavailable!\n[/]Processing may be slower.[/]")

print("\n" + "─" * 32 + "\n")

try:
	__modules = requirements["modules"]
	__packages = requirements["packages"]

	if __modules:
		for Name in __modules:
			!pip uninstall {Name} -y -q | grep -v "WARNING: Skip"
			!pip install -U {Name} --progress-bar off | grep -v "already"
			if all((len(__modules) > 1, Name != __modules[-1])):
				print()

	if __packages:
		for Name in __packages:
			!apt-get install --reinstall {Name} -qq > /dev/null 2 >&1
			if all((len(__packages) > 1, Name != __packages[-1])):
				print()

	import audioread
	from mutagen import File as mFile
	from hifi_gan_bwe import BandwidthExtender

	#-=-=-=-#
	# Requests

	session = Session()
	session.headers.update(
		{
			"User-Agent":
			"Mozilla/5.0 (Windows NT 10.0; Win32; x32) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
		}
	)

	#-=-=-=-#
	# Models

	print("\n" + "─" * 32 + "\n")
	Models = {
		"hifi-gan-bwe-10-42890e3-vctk-48kHz": """
			Same as [i]bwe-05[/], but [b]uses bandlimited interpolation for upsampling, [u]for reduced noise and aliasing[/][/].
			[u]Uses the same parameters as resampy's "kaiser_best" mode[/]
			""",
		"hifi-gan-bwe-11-d5f542d-vctk-8kHz-48kHz": "Same as [i]bwe-10[/], but [b]trained only on [i]8 000 Hz[/i][/] sources, [b u]for specialized upsampling[/]",
		"hifi-gan-bwe-12-b086d8b-vctk-16kHz-48kHz": "Same as [i]bwe-10[/], but [b]trained only on [i]16 000 Hz[/][/] sources, [b u]for specialized upsampling[/]",
		"hifi-gan-bwe-13-59f00ca-vctk-24kHz-48kHz": "Same as [i]bwe-10[/], but [b]trained only on [i]24 000 Hz[/][/] sources, [b u]for specialized upsampling[/]",
		"hifi-gan-bwe-05-cd9f4ca-vctk-48kHz": "Trained for 200K iterations on the VCTK [b u]speech dataset with noise agumentation[/] from the DNS Challenge dataset"
	}

	rmtree(os.path.expanduser("~/.local/hifi-gan-bwe"), True)
	for Key, Value in Models.items():
		Models[Key] = Value.replace("\t", "").strip("\n\t")
		BandwidthExtender.from_pretrained(Key)

	#-=-=-=-#
	# Functions

	def fdb_size(obj: str, extended_units: bool = False, bits: bool = False, recursive: bool = False) -> str:
		"""
		Args:
			obj: Bytes integer or string path of existing file or directory.
			extended_units: Extends the unit of the result, i.e. "Megabytes" instead of "MB".
			bits: Uses decimal divider (1000) instead of binary one. (1024)
			recursive: Iterate subdirectories, applicable only if `obj` is directory. (slow!)

		Returns:
			Human-readable files size string.
		"""
		# Setup

		if bits:
			Bits = 1000
			Bits_Display_Single = "bit"
			Bits_Display_Multiple = "bits"
		else:
			Bits = 1024
			Bits_Display_Single = "byte"
			Bits_Display_Multiple = "bytes"

		Units = {
			"":	 "",
			"K": "kilo",
			"M": "mega",
			"G": "giga",
			"T": "tera",
			"P": "peta",
			"E": "exa",
			"Z": "zetta",
			"Y": "yotta",
			"R": "ronna",
			"Q": "quetta"
		}

		#-=-=-=-#
		# Search for files

		if isinstance(obj, str):
			path = str(Path(obj).resolve())

			if not os.path.exists(path):
				raise FileNotFoundError(path)

			if os.path.isfile(obj):
				Files = [obj]
			else:
				Wildcard = "*"
				if recursive:
					Wildcard += "*/*"
				Files = list(Path(obj).glob(Wildcard))
				for File in Files:
					Files[Files.index(File)] = str(File.resolve())

			Files = map(os.path.getsize, Files)
			Bytes = sum(Files)
		else:
			Bytes = obj

		#-=-=-=-#
		# Calculate integer

		for Unit in Units:
			if Bytes < Bits:
				break
			Bytes /= Bits

		#-=-=-=-#
		# Ending conditions

		if extended_units:
			if Bytes == 1:
				Bits_Display = Bits_Display_Single
			else:
				Bits_Display = Bits_Display_Multiple
			Unit = (Units[Unit] + Bits_Display).capitalize()
		else:
			Unit += "B"

		#-=-=-=-#
		# Add zero to integer

		if "." in str(Bytes):
			Bytes = round(Bytes, 2)
			Bytes = str(Bytes).split(".")
			Bytes[1] = Bytes[1][:2]
			Bytes[1] = Bytes[1].ljust(2, "0")
			Bytes = ".".join(Bytes)

		Bytes = str(Bytes)

		#-=-=-=-#
		# Return string

		return " ".join((Bytes, Unit))

	#-=-=-=-#
	# Directories

	Directory_Main = os.path.abspath(__title__)
	Directory_Primary = os.path.join(Directory_Main, "Repository")
	Directory_Inputs = os.path.join(Directory_Main, "_inputs")
	Directory_Outputs = os.path.join(Directory_Main, "_outputs")

	if os.path.exists(Directory_Primary):
		rmtree(Directory_Primary)

	for Directory in Directory_Inputs, Directory_Outputs:
		os.makedirs(Directory, exist_ok = True)
except (KeyboardInterrupt, EOFError):
	print("\n" + "─" * 32 + "\n" + "Operation cancelled by user.")

# <div class="markdown-google-sans"><font color=gold>**2.**</font> Upload files 📤</div>

In [ ]:
#@markdown > 🗃️ <font color=lime>**<u>URL</u> of `ZIP` archive with files is supported.**

Direct_URL = "" #@param {type: "string"}
Allow_non_WAV_URL = True #@param {type: "boolean"}
__URL_Fail = False if Direct_URL else True

#@markdown ---

#@markdown > ℹ️ `Lazy Unpacking` determines whether <u>**files from the archive should only be unpacked if their file name extension**</u> is in the `Formats` variable.

Lazy_Unpacking = True #@param {type: "boolean"}
Formats = "AAC, AIFF, ALAC, APE, FLAC, M4A, MP3, OGG, OPUS, WAV, WV" #@param {type: "string"}
Formats = [Format.lower().strip() for Format in Formats.split(",")]

#@markdown ---

Ask_to_upload_files_if_download_succeeds = True #@param {type: "boolean"}
Remove_previously_uploaded_files_and_subdirectories = True #@param {type: "boolean"}

#-=-=-=-#

if "rmtree" not in locals():
	raise SystemExit("Requirements were not installed!\nPlease run the first cell.")

if "Files" not in locals():
	Files = []

#-=-=-=-#

__Directory_Base = os.getcwd()

# `os.chdir` since `files.upload` does not have the target directory argument
os.chdir(Directory_Inputs)

if Remove_previously_uploaded_files_and_subdirectories:
	Files_Inputs = []
	for File in Path(Directory_Inputs).rglob("*"):
		File = str(File.resolve())
		if is_colab:
			if ".ipynb_checkpoints" not in os.path.basename(File):
				Files_Inputs.append(File)
		else:
			File_Inputs.append(File)

	for File in Files_Inputs:
		if os.path.isfile(File):
			os.remove(File)
			rprint('Removed [b #44AAFF]"{0}"[/]'.format(
				os.path.relpath(File, Directory_Inputs)
			))

	for File in Files_Inputs:
		if os.path.isdir(File):
			rmtree(File)

	if Files_Inputs:
		print("\n" + "─" * 32 + "\n")

	Files = []

#-=-=-=-#

Allowed_Specifiers = "http", "https", "ftp"
Allowed_Specifiers = [Specifier + "://" for Specifier in Allowed_Specifiers]
Allowed_Specifiers = tuple(Allowed_Specifiers)

if Direct_URL.lower().startswith(Allowed_Specifiers):
	for Delimiter in (("?", "&", "#")):
		Downloaded_File_Location = Direct_URL.split(Delimiter)[0]

	Downloaded_File_Location = utils.unquote(Downloaded_File_Location.split("/")[-1])

	__URL_Fail = False
	try:
		Session_obj = session.get(Direct_URL, stream = 1)
	except Exception:
		__URL_Fail = True

	if __URL_Fail:
		rprint("[b #FF0000]Provided URL is invalid![/]")
		__URL_Fail = True

	if not __URL_Fail:
		with Session_obj as Site:
			if Site.ok:
				Site_MIME = Site.headers["content-type"].lower().split()[0].split(";")[0].split("/")
				if Site_MIME[0] == "audio":
					if any((Site_MIME[1].endswith("wav"), Allow_non_WAV_URL)):
						with open(Downloaded_File_Location, "wb") as File:
							rprint(f'Downloading [b #00CC00]"{Downloaded_File_Location}"[/]...')

							for Chunk in Site.iter_content(chunk_size = 1024):
								if Chunk:
									File.write(Chunk)
							Files.append(Downloaded_File_Location)
					else:
						rprint("[b #FF0000]Permission to download non-WAV files was not granted.[/]")
						__URL_Fail = True
				else:
					# ZIP Archive Handling
					if "/".join(Site_MIME) == "application/zip":
						with open(Downloaded_File_Location, "wb") as File:
							rprint(f'Downloading [b #00CC00]"{Downloaded_File_Location}"[/]...')

							for Chunk in Site.iter_content(chunk_size = 1024):
								if Chunk:
									File.write(Chunk)

						try:
							Archive_Files_Amount = 0
							with ZipFile(Downloaded_File_Location) as Archive:
								for Archive_File in Archive.infolist():
									if Lazy_Unpacking:
										if os.path.splitext(Archive_File.filename.lower())[1][1:] in Formats:
											Archive_Files_Amount += 1
											Archive.extract(Archive_File)
									else:
										Archive_Files_Amount += 1
										Archive.extract(Archive_File)

							os.remove(Downloaded_File_Location)

							#-=-=-=-#

							for File in Path().rglob("*.*"):
								File = str(File.resolve())

								if os.path.isfile(File):
									Files.append(File)

							rprint(f"[b #00CC00]{Archive_Files_Amount} files from main directory were extracted successfully![/]")
						except RuntimeError:
							rprint("[b #FF0000]Files extraction was not successful due to password requirement, file corruption or other internal error.[/]")
							__URL_Fail = True
					else:
						rprint("[b #FF0000]Provided URL is not an audio file.[/] [i #A0A0A0](undetected MIME type in header)[/]")
						__URL_Fail = True
			else:
				rprint("[b #FF0000]Failed to fetch the URL due to {0} status code. ({1})[/]".format(
					Site.status_code, Site.reason.title()
				))
				__URL_Fail = True

#-=-=-=-#
# Upload files

__Cancel_Permission = "" if __URL_Fail else " [i #A0A0A0](can be cancelled)[/]"

if any((Ask_to_upload_files_if_download_succeeds, __URL_Fail)):
	print("\n" + "─" * 32 + "\n")
	rprint(f"Upload [b #44AAFF]AUDIO[/] files{__Cancel_Permission}")

	try:
		for File in files.upload():
			Files.append(File)
	except TypeError:
		pass
	except (KeyboardInterrupt, EOFError):
		rprint("\n\n[b #FFCC00]File upload cancelled.[/]")

	Files = [str(Path(File).resolve()) for File in Files]
	if is_colab:
		Files = [File for File in Files if os.path.basename(File) != ".ipynb_checkpoints"]
	Files = list(set(Files))

#-=-=-=-#
# Check for files

Files = [os.path.abspath(File) for File in Files]

err = 0
try:
	# Use this later for string formatting
	Longest_File_Name_Length = [File.split(Directory_Inputs)[1][1:] for File in Files]
	Longest_File_Name_Length.sort(key = len)
	Longest_File_Name_Length = len(Longest_File_Name_Length[-1])
except IndexError:
	err = 1

if err:
	raise SystemExit("No uploaded files has been found.")

#-=-=-=-#

os.chdir(__Directory_Base)

In [ ]:
#@title <div class="markdown-google-sans">Verify files and convert them to `WAV` ⚙️</div>
#@markdown > ⚠️ **Files whose [MIME type](https://wikipedia.org/wiki/Media_type)'s first segment will not be detected as `audio` will be removed without warning.**
if "Files" not in locals():
	raise SystemExit("Requirements were not installed!\nPlease run the first cell.")

if not Files:
	raise SystemExit("No files has been found!")

#-=-=-=-#

Target = "WAV"

#-=-=-=-#

Target = Target.split()[0].upper().strip(".")

for Input in Files:
	Input_Split = os.path.split(os.path.abspath(Input))
	Input_Name = Input_Split[1]

	try:
		Input_MIME = mFile(Input).mime
	except AttributeError:
		Input_MIME = guess_type(Input)
	Input_MIME = Input_MIME[0].lower().split("/")

	if Input_MIME[0] == "audio":
		if Input_MIME[1] != Target.lower():
			os.environ["FILE_INPUT"] = Input
			os.environ["FILE_OUTPUT"] = os.path.splitext(Input)[0] + f".{Target.lower()}"

			rprint(f'[b #00CC00]"{Input_Name}" is not a {Target} file, converting...[/]')
			!ffmpeg -y -i "$FILE_INPUT" -hide_banner -loglevel 16 \
				-map_metadata 0 -map_metadata 0:s:0 \
				-fflags +bitexact -flags:v +bitexact -flags:a +bitexact \
				"$FILE_OUTPUT"

			Files[Files.index(Input)] = os.environ["FILE_OUTPUT"]
			os.remove(Input)
	else:
		rprint(f'[b #FF0000]"{Input_Name}" is not an audio file, removing[/]')
		Files.remove(Input)
		os.remove(Input)

#-=-=-=-#

if not Files:
	raise SystemExit("No files has been found!")

# <div class="markdown-google-sans"><font color=gold>**3.**</font> Process 📈</div>

In [ ]:
#@title <div class="markdown-google-sans">Load Model ⚙️</div>
#@markdown | Name | Quality (speech) | Quality (music) | Details |
#@markdown |:-|:-|:-|:-|
#@markdown | `hifi-gan-bwe-10-42890e3-vctk-48kHz` | ⭐ | ⭐ | Same as bwe-05, but uses bandlimited interpolation for upsampling, for reduced noise and aliasing.<br>Uses the same parameters as resampy's [`kaiser_best`](https://github.com/bmcfee/resampy/blob/5f46888e8b52402f2c62f374b39b93e0743543ad/resampy/filters.py#L9) mode. |
#@markdown | `hifi-gan-bwe-11-d5f542d-vctk-8kHz-48kHz` | ⭐⭐ | ⭐ | Same as bwe-10, but trained only on 8kHz sources, for specialized upsampling. |
#@markdown | `hifi-gan-bwe-12-b086d8b-vctk-16kHz-48kHz` | ⭐⭐⭐ | ⭐ | Same as bwe-10, but trained only on 16kHz sources, for specialized upsampling. |
#@markdown | `hifi-gan-bwe-13-59f00ca-vctk-24kHz-48kHz` | ⭐⭐⭐⭐ | ⭐⭐ | Same as bwe-10, but trained only on 24kHz sources, for specialized upsampling. |
#@markdown | `hifi-gan-bwe-05-cd9f4ca-vctk-48kHz` | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | Trained for 200K iterations on the VCTK speech dataset with noise agumentation from the DNS Challenge dataset. |

#@markdown ---

if "Models" not in locals():
	rprint("[b #FF0000]Requirements were not installed!\nPlease run the first cell.[/]")

Model_Name = "hifi-gan-bwe-05-cd9f4ca-vctk-48kHz" #@param ["hifi-gan-bwe-10-42890e3-vctk-48kHz", "hifi-gan-bwe-11-d5f542d-vctk-8kHz-48kHz", "hifi-gan-bwe-12-b086d8b-vctk-16kHz-48kHz", "hifi-gan-bwe-13-59f00ca-vctk-24kHz-48kHz", "hifi-gan-bwe-05-cd9f4ca-vctk-48kHz"]
Model = BandwidthExtender.from_pretrained(Model_Name)

#-=-=-=-#

rprint('[b][#00CC00]Successfully loaded "{0}" model.[/]'.format(Model_Name))
print()
rprint("\nDetails: {0}.".format(Models[Model_Name]))

In [ ]:
#@title # <div class="markdown-google-sans">**Run**</div>
#@markdown Select $\ge1$ output target samplerates.
SR_44100 = False #@param {type: "boolean"}
SR_48000 = True #@param {type: "boolean"}
SR_96000 = False #@param {type: "boolean"}
SR_192000 = False #@param {type: "boolean"}

SR_BASE = int(48_000)
Target_Samplerates = {
	44_100: SR_44100,
	SR_BASE: SR_48000,
	96_000: SR_96000,
	192_000: SR_192000
}

Sampling_Factors = {}
for Key in Target_Samplerates:
	if Key != SR_BASE:
		Sampling_Factors.update({Key: Key / SR_BASE})
Sampling_Factors.update({SR_BASE: 1})

Target_Samplerates = [int(Key) for Key, Value in Target_Samplerates.items() if Value]

if not Target_Samplerates:
	raise SystemExit("No target samplerates were selected!")

#@markdown > ⚠️ <font color=red>**Output target samplerates above `48 000 Hz`:**
#@markdown > - **Takes <u>longer to process</u>**
#@markdown > - **Uses <u>MUCH more RAM</u> and <u>may lead to the notebook failure!</u>**
#@markdown > - **Changes frequency balance**
#@markdown > - **Was tested <u>only on the `hifi-gan-bwe-05-cd9f4ca-vctk-48kHz` model</u>**</font>
#@markdown

#@markdown > ## ⚠️ <font color=red> **Previously processed files will be removed without warning!**</font>

#@markdown > ℹ️ <font color=gold> **Files that duration exceeds $10$ seconds are <u>not</u> recommended!**</font>

#@markdown ---

Sort_Files_by = "Upload order 🔢" #@param ["Upload order 🔢", "Duration 🕒", "Size ⚙️"]
Sort_Files_by = Sort_Files_by.lower()
Descending = False #@param {type: "boolean"}

#@markdown ---

Upscale_Internal_Gain = 0 # @param {type: "slider", min: 0, max: 6, step: 1}

#-=-=-=-#
# Files removal

Files_Outputs = []

for File in Path(Directory_Outputs).rglob("*"):
	File = str(File.resolve())
	if is_colab:
		if ".ipynb_checkpoints" not in os.path.basename(File):
			Files_Outputs.append(File)
	else:
		Files_Outputs.append(File)

for File in Files_Outputs:
	if os.path.isfile(File):
		os.remove(File)

for File in Files_Outputs:
	if os.path.isdir(File):
		rmtree(File)

#-=-=-=-#
# Sorting files

Files_Sorted = Files
if "duration" in Sort_Files_by:
	Files_Sorted = {File: mFile(File).info.length for File in Files}
	Files_Sorted = sorted(Files_Sorted.items(), key = lambda x: x[1])
	Files_Sorted = list(dict(Files_Sorted))
if "size" in Sort_Files_by:
	Files_Sorted = sorted(Files, key = os.path.getsize)

if Descending:
	Files_Sorted = Files_Sorted[::-1]

#-=-=-=-#

Runtime = time()
Counter_Samplerates = 1

torch.cuda.empty_cache()

try:
	with torch.no_grad():
		for Output_Target_Samplerate in Target_Samplerates:
			Output_Target_Samplerate_Display = f"{Output_Target_Samplerate:,}".split(",")
			Output_Target_Samplerate_Display = " ".join(Output_Target_Samplerate_Display) + " Hz"
			Counter_Samplerates_Display = str(Counter_Samplerates).rjust(len(str(len(Target_Samplerates))) + 1, "0")

			rprint("[b][#44AAFF]{0}.[/] Processing ([#44AAFF]{1}[/])[/]".format(
				Counter_Samplerates_Display,
				Output_Target_Samplerate_Display
			))

			Counter = 1
			for Input in Files_Sorted:
				Counter_Display = str(Counter).rjust(len(str(len(Files_Sorted))) + 1, "0")
				# Output file
				Input_Split = os.path.split(os.path.abspath(Input))

				Output = os.path.join(
					Directory_Outputs,
					os.path.relpath(Input_Split[0], Directory_Inputs),
					"_".join((
						os.path.splitext(Input_Split[1])[0],
						"scaled",
						str(Output_Target_Samplerate)
					)) + os.path.splitext(Input_Split[1])[1]
				)

				if not os.path.exists(os.path.dirname(Output)):
					os.makedirs(os.path.dirname(Output), exist_ok = True)

				#-=-=-=-#
				# Data

				with audioread.audio_open(Input) as File_Object:
					Samplerate = File_Object.samplerate
					Audio_Data = (
						np.hstack([np.frombuffer(b, dtype = np.int16) for b in File_Object])
						.reshape([-1, File_Object.channels])
						.astype(np.float32)
					) / (32767 - (Upscale_Internal_Gain * 1024))

				#-=-=-=-#
				# Scale

				File_Object = mFile(Input)
				MSR = Model.sample_rate

				rprint("    [b][#FFCC00]{0}.[/] [i]{1} [#A0A0A0]({2}s) ({3})[/][/][/]".format(
					Counter_Display,

					os.path.join(
						os.path.relpath(Input_Split[0], Directory_Inputs)[2:],
						Input_Split[1]
					).ljust(Longest_File_Name_Length, " "),

					round(File_Object.info.length, 3),
					fdb_size(Input)
				))

				# Main
				torch.cuda.empty_cache()
				Audio_Data = np.stack([
					Model(torch.from_numpy(x),
					int(Samplerate / Sampling_Factors[Output_Target_Samplerate])
					) for x in Audio_Data.T]).T
				write(
					Output,
					int(MSR * Sampling_Factors[Output_Target_Samplerate]),
					Audio_Data
				)

				Counter += 1

			#-=-=-=-#

			if all((len(Target_Samplerates) > 1, Output_Target_Samplerate != Target_Samplerates[-1])):
				print()

			Counter_Samplerates += 1
except (KeyboardInterrupt, EOFError):
	rprint("[#FF0000]Operation cancelled by user.[/]")

torch.cuda.empty_cache()

#-=-=-=-#

__Run_Time = time() - Runtime
print("\n" + "─" * 32 + "\n")
rprint("[b #00CC00]Done![/]")
rprint("Runtime: [b #44AAFF]{0}[/]".format(
	str(timedelta(seconds = __Run_Time))[2:-3]
))
rprint("\t[i #A0A0A0]avg. {0} per file[/]".format(
	str(timedelta(seconds = __Run_Time / len(Files)))[2:-3]
))
rprint("\t[i #A0A0A0]avg. {0} per samplerate[/]".format(
	str(timedelta(seconds = (__Run_Time / len(Files)) / len(Target_Samplerates)))[2:-3]
))

# <div class="markdown-google-sans">Utils ⚙️</div>

In [ ]:
#@title <div class="markdown-google-sans">Convert output files to `FLAC` 🎶</div>

Bit_Depth = "24-bit" #@param ["16-bit", "24-bit"]
Bit_Depth = Bit_Depth.split("-")[0]
Normalize_dB = 1 # @param {type: "slider", min: -6, max: 1, step: 1}
#@markdown > ℹ️ To <font color=gold>**disable**</font> normalization, set `Normalize_dB` to **`1`**.

#-=-=-=-#

Target = "FLAC"
Target = Target.split()[0].lower().strip(".")

#-=-=-=-#

os.environ["BIT_DEPTH"] = "-b " + Bit_Depth
os.environ["NORMALIZE"] = ("--norm " + str(Normalize_dB)) if Normalize_dB < 1 else ""

for File in Path(Directory_Outputs).rglob("*.*"):
	Input = str(File.resolve())

	if os.path.isfile(Input):
		Input_MIME = mFile(Input).mime[0].lower().split("/")
		Input_Name = os.path.relpath(Input, Directory_Main)

		Output_FLAC = os.path.splitext(Input)[0], f".{Target.lower()}"
		Output = "_".join(Output_FLAC)

		os.environ["FILE_INPUT"] = Input
		os.environ["FILE_OUTPUT"] = Output

		rprint('[b i]{1}onverting [#44AAFF]"{0}"[/] to FLAC...[/]'.format(
			Input_Name,
			("rec" if Input_MIME[1] == "flac" else "c").capitalize()
		))
		!sox "$FILE_INPUT" -D $BIT_DEPTH -C 8 $NORMALIZE --comment "" -G "$FILE_OUTPUT" -V0

		os.remove(Input)
		os.rename(Output, "".join(Output_FLAC))

In [ ]:
#@title <div class="markdown-google-sans">Download archive with output files 🗃️</div>
Emphasize_Archive_File_Name = True #@param {type: "boolean"}
Beggining = "!_" if Emphasize_Archive_File_Name else ""

#@markdown > ℹ️ Pushes the archive file to top of the explorer's file list (**if** it's sorted by name) by putting `!_` to its file name beggining.

#-=-=-=-#

__Padding = 7
Archive_File = Beggining + "_".join((__title__, str(time()).split(".")[1][:__Padding].rjust(__Padding, "0")))
Archive_File += ".zip"
Archive_File = os.path.abspath(Archive_File)

#-=-=-=-#

# Archive Creation
print("Making Archive...")
with ZipFile(Archive_File, "w", 14, 1, 9) as Archive:
	for File in Path(Directory_Outputs).rglob("*.*"):
		File = str(File.resolve())

		if os.path.isfile(File):
			ArcName = os.path.relpath(File, Directory_Outputs)

			rprint(f'\t[b i]Writing [#44AACC]"{ArcName}"[/]...')
			Archive.write(File, ArcName)

#-=-=-=-#

files.download(Archive_File)

In [ ]:
#@title <font class="markdown-google-sans">Terminate runtime after specified time</font> 🔌
#@markdown In case if you don't want to waste this account's resources while performing other tasks.
#@markdown > ### <font color=red>**This <font color=gold>cancellable</font> action is irreversible and will result in loss of ALL session data!**</font>

#@markdown <br>
#@markdown <details open>
#@markdown     <summary>Time sheet</summary>
#@markdown
#@markdown <br>
#@markdown
#@markdown | Seconds | Minutes |
#@markdown |:-|:-|
#@markdown | `60` | `1` |
#@markdown | `300` | `5` |
#@markdown | `600` | `10` |
#@markdown | `1800` | `30` |
#@markdown | `3600` | `60` |
#@markdown </details>

#@markdown ---

Seconds_Default = 30
Seconds = 300 # @param {type: "integer"}

if Seconds < 0:
	rprint(f"[b][#FFCC00]Seconds are[/] [#FF0000]< 0[/][#FFCC00], defaulting to[/] [#FF0000]{Seconds_Default}[/][#FFCC00].[/][/]")

__Spaces = " " * 8



if is_colab:
	try:
		for i in range(Seconds, -1, -1):
			print("\r", end = f"Runtime will be terminated in {i} seconds..." + __Spaces)

		rprint("\n\n[b #FF0000]Runtime has been terminated.[/]")
		runtime.unassign()
	except (KeyboardInterrupt, EOFError):
		print()
		rprint("[b #FF0000]Operation cancelled by user.[/]")
else:
	rprint("[b #FFCC00]Not in Colab, unsupported.[/]")